In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
mnist = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv')
mnist.describe()

In [ ]:
import matplotlib.pyplot as plt

def display(image):
    plt.imshow(image)
    plt.show()

In [ ]:
mnist = mnist.values
x = mnist[:,1:]
y = mnist[:,0]
print(x.shape,y.shape)

In [ ]:
idx = 3049
sample = x[idx]
sample = sample.reshape((28,28))
display(sample)
print(y[idx])

In [ ]:
print(np.max(sample))

In [ ]:
x = x/255

In [ ]:
x = x.reshape((-1,28,28))

In [ ]:
y = y.reshape((-1,1))

In [ ]:
print(x.shape,y.shape)

In [ ]:
import torch
from torch import nn,optim
from torch.utils.data import DataLoader,TensorDataset

In [ ]:
x = torch.tensor(x).float()
y = torch.tensor(y).float()
print(x.shape,y.shape)

In [ ]:
dataset = TensorDataset(x,y)
dataloader = DataLoader(dataset,batch_size = 128,shuffle=True)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator,self).__init__()
        n_features = 784
        n_out = 1
        
        self.layers = nn.Sequential(
            nn.Linear(n_features,1024),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(1024,512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512,256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(256,n_out),
            nn.Sigmoid()
        )    
    def forward(self,x):
        return self.layers(x)

discriminator = Discriminator()

In [ ]:
def images_to_vector(images):
    return images.view(images.size(0),784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0),1,28,28)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        n_features = 100
        n_out = 784
        
        self.layers = nn.Sequential(
            nn.Linear(n_features,256),
            nn.LeakyReLU(0.2),
            
            nn.Linear(256,512),
            nn.LeakyReLU(0.2),
            
            nn.Linear(512,1024),
            nn.LeakyReLU(0.2),
            
            nn.Linear(1024,n_out),
            nn.Sigmoid()
        )    
    def forward(self,x):
        return self.layers(x)

generator = Generator()

In [ ]:
from torch.autograd.variable import Variable
def noise(size):
    return Variable(torch.randn(size,100))

In [ ]:
d_optim = optim.Adam(discriminator.parameters(),lr=0.0002)
g_optim = optim.Adam(generator.parameters(),lr=0.0002)

In [ ]:
loss = nn.BCELoss()

In [ ]:
def ones_target(size):
    return Variable(torch.ones(size,1))
def zeros_target(size):
    return Variable(torch.zeros(size,1))

In [ ]:
def train_discriminator(optimizer,real_data,fake_data):
    N = real_data.size(0)
    optimizer.zero_grad()
    
    pred_real = discriminator(real_data)
    error_real = loss(pred_real,ones_target(N))
    error_real.backward()
    
    pred_fake = discriminator(fake_data)
    error_fake = loss(pred_fake,zeros_target(N))
    error_fake.backward()
    
    optimizer.step()
    
    return error_real + error_fake, pred_real,pred_fake

In [ ]:
def train_generator(optimizer,fake_data):
    N = fake_data.size(0)
    optimizer.zero_grad()
    
    pred = discriminator(fake_data)
    error = loss(pred,ones_target(N))
    error.backward()
    optimizer.step()
    return error

In [ ]:
def fsample(images,epoch,rows,cols):        
    fig,axs = plt.subplots(rows,cols,figsize=(8,3))
    cnt  = 0
    for i in range(rows):
        for j in range(cols):
            axs[i,j].imshow(images[cnt],cmap ='gray')
            axs[i,j].axis('off')
            cnt += 1 
    plt.show()
    plt.close()

In [ ]:
epochs = 20
test_noise = noise(16)

for epoch in range(epochs):
    for n_batch,(real_batch,_) in enumerate(dataloader):
        N = real_batch.size(0)
        
        real_data = images_to_vector(real_batch)
        
        fake_data  = generator(noise(N))
        
        #D training
        d_error,d_pred_real,d_pred_fake = train_discriminator(d_optim,real_data,fake_data)
        
        #G traiing
        fake_data = generator(noise(N))
        
        g_error = train_generator(g_optim,fake_data)
        
    if (epoch) % 100 == 0:
        if 1==1:
            test_images = vectors_to_images(generator(test_noise))
            test_images = test_images.data
            test_images = test_images.numpy().reshape((-1,28,28))
            fsample(test_images,epoch,4,4)
            
        
torch.save(discriminator.state_dict(), './')
torch.save(generator.state_dict(), './')
